In [4]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import torchaudio
from pydub import AudioSegment

input_path = "./Recording.m4a"

# Wczytaj plik .m4a
audio = AudioSegment.from_file(input_path, format="m4a")

# Konwertuj do mono i 16kHz
audio = audio.set_channels(1)
audio = audio.set_frame_rate(16000)

# Zapisz jako .wav
audio.export("sample.wav", format="wav")
# Ścieżka do pliku .m4a

# Załaduj i przekonwertuj do mono i 16kHz
waveform, sample_rate = torchaudio.load("sample.wav")

# Jeśli stereo, zamień na mono przez uśrednienie kanałów
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

# Resampling do 16kHz jeśli trzeba
if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)
    sample_rate = 16000

# Squeeze do (audio_length,)
waveform = waveform.squeeze()

# Inicjalizacja modelu i procesora
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Przygotowanie wejścia
inputs = processor(waveform, sampling_rate=sample_rate, return_tensors="pt", padding=True)

# Przewidywanie
with torch.no_grad():
    logits = model(**inputs).logits

# Dekodowanie do tekstu
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
print(transcription)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['HALLO']


In [5]:
predicted_ids

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 11, 11,  0,  0,
          0,  0,  0,  7, 15, 15,  0,  0,  0, 15,  0,  0,  0,  0,  0,  0,  0,  8,
          8,  8,  0,  0,  0,  4,  4,  4,  4,  4,  4,  4,  4,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])